In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pickle

# Load the data
file_paths = [
    'bag_toss_labeled_1.csv',
    'sitting_up_down_labeled_2.csv',
    'stairs_labeled_4.csv',
    'walking_labeled_3.csv'
]
data_frames = [pd.read_csv(file_path) for file_path in file_paths]
combined_data = pd.concat(data_frames)

# Preprocess the data
data_for_model = combined_data.drop(columns=['timestamp'])  # Assuming timestamp is not used
X = data_for_model.drop(columns=['label'])
y = data_for_model['label'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

X_train_lstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=True),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
model.fit(X_train_lstm, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping], batch_size=64)

# Save the model and preprocessors
model.save('lstm_model.h5')
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

print("Model and preprocessors saved successfully.")





Epoch 1/50


285/285 [==============================] - 5s 7ms/step - loss: 1.1380 - accuracy: 0.5390 - val_loss: 0.9094 - val_accuracy: 0.6357
Epoch 2/50
285/285 [==============================] - 1s 4ms/step - loss: 0.7971 - accuracy: 0.6813 - val_loss: 0.7863 - val_accuracy: 0.6735
Epoch 3/50
285/285 [==============================] - 1s 4ms/step - loss: 0.7152 - accuracy: 0.7039 - val_loss: 0.7419 - val_accuracy: 0.6840
Epoch 4/50
285/285 [==============================] - 1s 4ms/step - loss: 0.6865 - accuracy: 0.7113 - val_loss: 0.7266 - val_accuracy: 0.6878
Epoch 5/50
285/285 [==============================] - 1s 4ms/step - loss: 0.6682 - accuracy: 0.7164 - val_loss: 0.7101 - val_accuracy: 0.6961
Epoch 6/50
285/285 [==============================] - 1s 4ms/step - loss: 0.6533 - accuracy: 0.7231 - val_loss: 0.6963 - val_accuracy: 0.6983
Epoch 7/50
285/285 [==============================] - 1s 4ms/step - loss: 0.6413 - accuracy: 0.7290 - val_loss: 0.6888 - val_accuracy: 0.7058
E

c:\Users\jakok\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
